## Notes

* should examine some of the LEAST agricultural districts in contrast to the ~60 MOST (which I have).

# Summary:

This work follows Bruckner and Ciccone (2011), who found that the negative exogenous economic shocks which followed droughts in sub-Saharan Africa were solid contributors to / predictors of democratic revolutions. 

I attempt to establish a relationship between droughts (again serving as negative exogenous economic shocks) and political change, which might take any of the following forms:
* Increased votes for a specific pro-farmer party (will have to investigate whether democrats or republicans are viewed as pro-farmer.)
* Increased probability of an incumbent losing
* Increased votes for the Democratic party, who generally favor greater redistribution (see: Acemoglu & Robinson)

  
  
 newline

The equation of interest takes the form \begin{equation} Y_{i,t} = \alpha \, * Drought_{i,t} + \beta'X_{i,t} + \phi_{i,t} + \psi_{i,t} + u_{i,t} \end{equation}

NOTE: will need to include (and interact) a dummy term for presidential election or not that year.

where 
* $Y_{i,t} = $ our outcome of interest - either vote share for a particular party; probability of an incumbent losing; etc.
* $\alpha = $ the effect of drought on the outcome of interest
* $X_{i,t} = $ characteristics of the district at time *t*, such as income, farming as a share of GDP, average farm size, etc.
* $\phi_{i,t} = $ district fixed effects
* $\psi_{i,t} = $ time fixed effects
* state fixed effects?

include willingness to elect other party? years since change of power? avg years for incumbent to stay in power?

# Data Import and Cleaning

## Import and Setup

In [50]:
import pandas as pd
import numpy as np

In [51]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Market Value by District Data

Note: Due to data/time limitations, I'm assuming that congressional districts that are currently highly agricultural were also highly agricultural over the period I examine. This is one place to expand my analysis when I have more time

In [52]:
# from 
# https://www.nass.usda.gov/Publications/AgCensus/2012/Online_Resources/Congressional_District_Rankings/ , and
# https://www.nass.usda.gov/Publications/AgCensus/2017/Online_Resources/Congressional_District_Rankings/
farm_census_2012 = pd.read_csv(r"C:\Users\mikha\OneDrive\Desktop\Dropbox\MIKHAEL NEW\mikhael school\Grad School\Master's\Term 2 Classes\544\Replication\My Addition\Mkt Value of Agr Products Sold\2012.csv")
farm_census_2017 = pd.read_csv(r"C:\Users\mikha\OneDrive\Desktop\Dropbox\MIKHAEL NEW\mikhael school\Grad School\Master's\Term 2 Classes\544\Replication\My Addition\Mkt Value of Agr Products Sold\2017.csv")

farm_census_2012['year'] = 2012
farm_census_2017['year'] = 2017

# merge 2012 and 2017
farm_census = pd.concat([farm_census_2012, farm_census_2017])

del farm_census_2012, farm_census_2017

### Some Text Cleaning

In [53]:
# upper-case
farm_census['state'] = [x.upper() for x in farm_census['state']]

#remove non-alphabet characters
farm_census['state'] = farm_census['state'].str.replace('\d+', '')
farm_census['state'] = farm_census['state'].str.replace('/', '')

In [54]:
# narrow down districts we can focus the analysis on
usable_districts = {state:set() for state in farm_census['state']}

In [55]:
for state_dist in list(zip(farm_census['state'], farm_census['district'])):
    if state_dist[1] != 'At Large':
        usable_districts[state_dist[0]].add(int(state_dist[1]))
    else:
        usable_districts[state_dist[0]].add(state_dist[1])

## Election Data (House of Rep's)

Include Senate data (same source)? Only catch is it isn't as specific to the agricultural districts. Could still be useful as a baseline.

**TO DO**
* Denote who won each election
    * Create "years in seat" variable
* Figure out whether to use pre-1998 data for incumbency
* Figure out "At Large" stuff
    * Note: "At Large" districts arenare denoted with zeros
* There are "NaNs" in the data (under candidate name)

In [56]:
# from https://electionlab.mit.edu/data 
elec_df = pd.read_csv(r"C:\Users\mikha\OneDrive\Desktop\Dropbox\MIKHAEL NEW\mikhael school\Grad School\Master's\Term 2 Classes\544\Replication\My Addition\1976-2018 US House Election Data.csv", engine='python')

### Drop Unecessary Data Features and Observations

In [58]:
# #drop if before 1998
# elec_df = elec_df[elec_df['year'] >= 1998]

#drop useless columns (one unique value and/or irrelevant info)
elec_df.drop(['office', 'mode', 'version'], axis=1, inplace=True)

#drop if not general election (other possibilites are primary or blank). This only drops ~90/30000 obs
elec_df = elec_df[elec_df['stage']=='gen']

# drop runoffs - only 8 obs. Note that ~9k / 30k obs are "NA" under "runoff"... data description doesn't say what this means
elec_df = elec_df[elec_df['runoff']!='TRUE']

### Denote presidential election years

In [59]:
elec_df['DPres'] = elec_df['year'] % 4 == 0

### Denote election observations corresponding to *agricultural* districts

In [60]:
# DENOTE WHETHER OR NOT IS A BIG AGRICULTURAL DISTRICT (that we have economic/agricultural data for)

# this loop just appends 0 or 1 to the vector "usable_obs" if we have economic/agricultural data for that district
usable_obs = []

for i, row in elec_df.iterrows():
    if row['state'] in usable_districts and row['district'] in usable_districts[row['state']]:
        usable_obs.append(1)
    else:
        usable_obs.append(0)

# here we bring this vector into the df        
elec_df['usable'] = usable_obs

del usable_obs


# COULD drop all observations that don't correspond to "usable" districts. but probably best to keep them in the df for now
# This would leave us with 2974 election observations, before narrowing it down to winners
# elec_df = elec_df[elec_df['usable']==1]

### Who won, and by how much? Incumbent? Year on year change?

**NEED TO ADD MARGIN OF VICTORY, AND MARGIN OF VICTORY COMPARED TO LAST TIME**

In [61]:
#voting dict is structured as voting_dict[state][year][district][candidate][votes]

def div_by_zero(x,y):
    if y!=0:
        return x/y
    else:
        return 0

voting_dict = {}
for state in set(elec_df['state']):
    voting_dict[state] = {}
    for year in set(elec_df['year']):
        voting_dict[state][year] = {}



#do NOT combine following two loops. Otherwise, dict entries are overwritten for every new line that's iterated over
for i, row in elec_df.iterrows():
    voting_dict[ row['state'] ][ row['year'] ][ row['district'] ] = {}

for i, row in elec_df.iterrows():
    voting_dict[ row['state'] ][ row['year'] ][ row['district'] ] [row['candidate']] = {'candidatevotes':row['candidatevotes'], 'totalvotes':row['totalvotes'], 'voteshare':div_by_zero(row['candidatevotes'],row['totalvotes'])}

    
    
#find winner of each election
for state in voting_dict:
    for year in voting_dict[state]:
        for district in voting_dict[state][year]:
            for candidate in voting_dict[state][year][district]:
                if voting_dict[state][year][district][candidate]['voteshare'] == max([voting_dict[state][year][district][can]['voteshare'] for can in voting_dict[state][year][district]]):
                    voting_dict[state][year][district][candidate]['winner'] = 1
                else:
                    voting_dict[state][year][district][candidate]['winner'] = 0
                    
                    
# find change in voting share from last election. also create "incumbent" field
for state in voting_dict:
    for year in voting_dict[state]:
        for district in voting_dict[state][year]:
            for candidate in voting_dict[state][year][district]:
                try: #here I subtract voteshare this year from voteshare in the last election (2 years prior). if this is their first election, var set to nan
                    voting_dict[state][year][district][candidate]['votesharediff'] = voting_dict[state][year][district][candidate]['voteshare'] - voting_dict[state][year - 2][district][candidate]['voteshare']
                    voting_dict[state][year][district][candidate]['incumbent'] = 1
                except:
                    voting_dict[state][year][district][candidate]['votesharediff'] = np.nan
                    voting_dict[state][year][district][candidate]['incumbent'] = 0
                    
# find total election turnout
for state in voting_dict:
    for year in voting_dict[state]:
        for district in voting_dict[state][year]:
            for candidate in voting_dict[state][year][district]:
                voting_dict[state][year][district][candidate]['total turnout'] = sum([voting_dict[state][year][district][can]['totalvotes'] for can in voting_dict[state][year][district]])



### Bring "voting_dict" info into elec_df

In [62]:
winner = []
incumbent = []
share_diff = []
voteshare = []
totalturnout = []

for i, row in elec_df.iterrows():
    winner.append(voting_dict[row['state']][row['year']][row['district']][row['candidate']]['winner'])
    incumbent.append(voting_dict[row['state']][row['year']][row['district']][row['candidate']]['incumbent'])
    share_diff.append(voting_dict[row['state']][row['year']][row['district']][row['candidate']]['votesharediff'])
    voteshare.append(voting_dict[row['state']][row['year']][row['district']][row['candidate']]['voteshare'])
    totalturnout.append(voting_dict[row['state']][row['year']][row['district']][row['candidate']]['totalvotes'])

    
elec_df['winner'] = winner
elec_df['incumbent'] = incumbent
elec_df['share_diff'] = share_diff
elec_df['voteshare'] = voteshare
elec_df['totalturnout'] = totalturnout

### Crucial to drop non-major parties *after* creating "voting_dict"

Otherwise vote share will be incorrectly totalled

In [63]:
#drop if not Democratic, Republican, or Libertarian candidate
elec_df = elec_df[elec_df['party'].isin(['DEMOCRAT','REPUBLICAN', 'LIBERTARIAN'])]

## Drought Data

In [76]:
# from https://droughtmonitor.unl.edu/Data/DataDownload/ComprehensiveStatistics.aspx
# data description here (under "Excel") https://droughtmonitor.unl.edu/Data/Metadata.aspx
drought_df = pd.read_csv(r"C:\Users\mikha\OneDrive\Desktop\Dropbox\MIKHAEL NEW\mikhael school\Grad School\Master's\Term 2 Classes\544\Replication\My Addition\Drought Data.csv")

### Create "year" var

In [77]:
drought_df['year'] = drought_df['ValidStart'].astype(str).str[:4].astype(int)

### Drop some redundant fields

In [78]:
drought_df.drop(['MapDate', 'FIPS'], axis=1, inplace=True)

### Take yearly averages per county

In [75]:
drought_df = drought_df.groupby(['State', 'County', 'year']).mean()
drought_df

None     D0     D1    D2   D3   D4  \
State County                 year                                        
AK    Aleutians East Borough 2000  99.40   0.60   0.00  0.00 0.00 0.00   
                             2001 100.00   0.00   0.00  0.00 0.00 0.00   
                             2002 100.00   0.00   0.00  0.00 0.00 0.00   
                             2003 100.00   0.00   0.00  0.00 0.00 0.00   
                             2004 100.00   0.00   0.00  0.00 0.00 0.00   
...                                  ...    ...    ...   ...  ...  ...   
WY    Weston County          2017  30.26  69.74  32.44  1.41 0.00 0.00   
                             2018  88.22  11.78   0.00  0.00 0.00 0.00   
                             2019  99.98   0.02   0.00  0.00 0.00 0.00   
                             2020  40.38  59.62  48.60  1.99 0.00 0.00   
                             2021   0.00 100.00 100.00 53.55 0.00 0.00   

                                   StatisticFormatID  
State County                 year                     
AK    Aleutians East Borough 2000                  1  
                             2001                  1  
                             2002                  1  
                             2003                  1  
                             2004                  1  
...                                              ...  
WY    Weston County          2017                  1  
                             2018                  1  
                             2019                  1  
                             2020                  1  
                             2021                  1  

[70818 rows x 7 columns]

In [74]:
drought_df

None     D0     D1    D2   D3   D4  \
State County                 year                                        
AK    Aleutians East Borough 2000  99.40   0.60   0.00  0.00 0.00 0.00   
                             2001 100.00   0.00   0.00  0.00 0.00 0.00   
                             2002 100.00   0.00   0.00  0.00 0.00 0.00   
                             2003 100.00   0.00   0.00  0.00 0.00 0.00   
                             2004 100.00   0.00   0.00  0.00 0.00 0.00   
...                                  ...    ...    ...   ...  ...  ...   
WY    Weston County          2017  30.26  69.74  32.44  1.41 0.00 0.00   
                             2018  88.22  11.78   0.00  0.00 0.00 0.00   
                             2019  99.98   0.02   0.00  0.00 0.00 0.00   
                             2020  40.38  59.62  48.60  1.99 0.00 0.00   
                             2021   0.00 100.00 100.00 53.55 0.00 0.00   

                                   StatisticFormatID  
State County                 year                     
AK    Aleutians East Borough 2000                  1  
                             2001                  1  
                             2002                  1  
                             2003                  1  
                             2004                  1  
...                                              ...  
WY    Weston County          2017                  1  
                             2018                  1  
                             2019                  1  
                             2020                  1  
                             2021                  1  

[70818 rows x 7 columns]

## Mapping Files

## Congressional District Shapefiles

source: http://cdmaps.polisci.ucla.edu/

In [83]:
conda install pandas fiona shapely pyproj rtree

Note: you may need to restart the kernel to use updated packages.



Examining conda-env:   1%|          | 2/312 [00:00<00:03, 96.24it/s]



Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed


Examining ipython_genutils: 100%|##########| 312/312 [03:03<00:00,  1.34it/s]
                                                                             

Determining conflicts:   0%|          | 0/312 [00:00<?, ?it/s]
Examining conflict for diff-match-patch spyder anaconda:   0%|          | 0/312 [00:00<?, ?it/s]
Examining conflict for pandas diff-match-patch mkl_fft olefile libtiff tk widgetsnbextension anaconda-project bzip2 defusedxml xlwt json5 openssl liblief dask-core bokeh nose backports.shutil_get_terminal_size asn1crypto bcrypt ujson babel pexpect nbconvert nbformat chardet fsspec pyodbc heapdict jupyter_core prometheus_client werkzeug unicodecsv cffi cycler itsdangerous importlib-metadata atomicwrites conda-build spyder-kernels webencodings backports.weakref zope.interface pydocstyle pickleshare pynacl singledispatch pywinpty partd typing_extensions backports jpeg bkcharts urllib3 greenlet ipykernel xlrd jupyterlab_pygments pyparsing sphinxcontrib-websupport pyrsistent flak

Examining conflict for typing_extensions python-language-server spyder numpydoc flake8 bokeh sphinx:  50%|####9     | 155/312 [17:19<15:23,  5.88s/it]
Examining conflict for typing_extensions python-language-server spyder numpydoc flake8 bokeh sphinx:  50%|#####     | 156/312 [17:19<15:31,  5.97s/it]
Examining conflict for watchdog argh spyder anaconda:  50%|#####     | 156/312 [17:20<15:31,  5.97s/it]                                               
Examining conflict for watchdog argh spyder anaconda:  50%|#####     | 157/312 [17:20<11:46,  4.56s/it]
Examining conflict for sphinxcontrib-applehelp spyder anaconda numpydoc sphinx:  50%|#####     | 157/312 [17:27<11:46,  4.56s/it]
Examining conflict for sphinxcontrib-applehelp spyder anaconda numpydoc sphinx:  51%|#####     | 158/312 [17:27<12:53,  5.02s/it]
Examining conflict for numba jupyter_client matplotlib-base anaconda-project bokeh pylint nltk notebook singledispatch dask tornado ipykernel anaconda jupyterlab astroid python-langua

...


Examining libspatialindex: 100%|#########9| 309/310 [13:44<00:01,  1.43s/it]
                                                                            

Determining conflicts:   0%|          | 0/310 [00:00<?, ?it/s]
Examining conflict for get_terminal_size anaconda:   0%|          | 0/310 [00:00<?, ?it/s]
Examining conflict for colorama get_terminal_size zope.event python-jsonrpc-server spyder zipp joblib pyyaml nltk pytz mkl_random diff-match-patch prometheus_client attrs bokeh pyodbc xlsxwriter wincertstore notebook conda kiwisolver sphinxcontrib-applehelp idna ujson sphinxcontrib-devhelp keyring pyzmq bkcharts pylint numexpr unicodecsv clyent sortedcollections itsdangerous conda-package-handling msgpack-python babel jedi pywin32-ctypes widgetsnbextension cytoolz anaconda-client qtawesome backcall setuptools pip sphinxcontrib-websupport anaconda-navigator cycler olefile nbclient bcrypt qdarkstyle astroid autopep8 scikit-learn dask-core matplotlib-base pywinpty jupyter_client anaco

seaborn -> statsmodels[version='>=0.8.0'] -> vc[version='10.*|14.*|9.*|>=14,<15.0a0|>=14.1,<15.0a0|>=9,<10.0a0']
scikit-learn -> vc[version='14.*|>=14,<15.0a0|>=14.1,<15.0a0|9.*']
mkl-service -> python[version='>=2.7,<2.8.0a0'] -> vc[version='>=9,<10.0a0']
itsdangerous -> python -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0|9.*']
qt -> zlib[version='>=1.2.11,<1.3.0a0'] -> vc[version='10.*|14|9|>=14,<15.0a0|>=9,<10.0a0']
pyrsistent -> python[version='>=2.7,<2.8.0a0'] -> vc[version='>=9,<10.0a0']
backports.functools_lru_cache -> python -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0|9.*']
sphinxcontrib-websupport -> python -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0|9.*']
prompt_toolkit -> python[version='>=3.6'] -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0|9.*']
pep8 -> python -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0|9.*']
pycurl -> python[version='>=2.7,<2.8.0a0'] -> vc[version='10.*|14

networkx -> setuptools -> certifi[version='>=2016.09|>=2016.9.26']
isort -> setuptools -> certifi[version='>=2016.09|>=2016.9.26']
jupyter_client -> tornado[version='>=4.1'] -> certifi
jupyterlab -> tornado[version='!=6.0.0,!=6.0.1,!=6.0.2'] -> certifi
matplotlib-base -> certifi[version='>=2020.06.20']
jupyterlab_server -> requests -> certifi[version='>=2017.4.17']
bleach -> setuptools -> certifi[version='>=2016.09|>=2016.9.26']
numba -> setuptools -> certifi[version='>=2016.09|>=2016.9.26']
pip -> setuptools -> certifi[version='>=2016.09|>=2016.9.26|>=2017.4.17']
urllib3 -> certifi
setuptools -> certifi[version='>=2016.09|>=2016.9.26']
ruamel_yaml -> setuptools -> certifi[version='>=2016.09|>=2016.9.26']
clyent -> setuptools -> certifi[version='>=2016.09|>=2016.9.26']
nose -> setuptools -> certifi[version='>=2016.09|>=2016.9.26']
bokeh -> tornado[version='>=4.3'] -> certifi[version='>=2017.4.17']
anaconda-project -> requests -> certifi[version='>=2017.4.17']
pywinpty -> setuptools -> 

jupyter -> notebook -> ipaddress
pyopenssl -> cryptography[version='>=2.8'] -> ipaddress
ipywidgets -> notebook -> ipaddress
widgetsnbextension -> notebook[version='>=4.4.1'] -> ipaddress
jupyterlab_server -> notebook[version='>=4.2.0'] -> ipaddress
notebook -> ipaddress
cryptography -> ipaddress

Package pytest-cov conflicts for:
geopandas -> pysal -> pytest-cov
astropy -> pytest-astropy -> pytest-cov[version='>=2.0']

Package bleach conflicts for:
notebook -> nbconvert -> bleach
jupyter -> nbconvert -> bleach
anaconda==2020.11 -> nbconvert==6.0.7=py37_0 -> bleach
nbconvert -> bleach
spyder -> nbconvert[version='>=4.0'] -> bleach
anaconda==2020.11 -> bleach==3.2.1=py_0

Package olefile conflicts for:
bokeh -> pillow[version='>=7.1.0'] -> olefile
scikit-image -> pillow[version='>=4.3.0'] -> olefile
pillow -> olefile
imageio -> pillow -> olefile
anaconda-navigator -> pillow -> olefile
matplotlib-base -> pillow[version='>=6.2.0'] -> olefile
anaconda==2020.11 -> olefile==0.46[build='py_0|

Package tbb4py conflicts for:
mkl_fft -> numpy-base[version='>=1.0.6,<2.0a0'] -> tbb4py
numpy-base -> tbb4py
numpy -> numpy-base==1.15.4=py27h2753ae9_0 -> tbb4py
mkl_random -> numpy-base[version='>=1.0.2,<2.0a0'] -> tbb4py

Package send2trash conflicts for:
anaconda==2020.11 -> notebook==6.1.4=py37_0 -> send2trash
jupyter -> notebook -> send2trash[version='>=1.5.0']
anaconda==2020.11 -> send2trash==1.5.0[build='py37_0|py38_0']
jupyterlab -> jupyter_server[version='>=1.4,<2'] -> send2trash[version='>=1.5.0']
widgetsnbextension -> notebook[version='>=4.4.1'] -> send2trash[version='>=1.5.0']
ipywidgets -> notebook -> send2trash[version='>=1.5.0']
notebook -> send2trash[version='>=1.5.0']
jupyterlab_server -> jupyter_server[version='>=1.1.0,<2.0.0'] -> send2trash[version='>=1.5.0']

Package astroid conflicts for:
spyder -> pylint[version='>=1.0'] -> astroid[version='>=1.4.5,<1.5.0|>=1.5.1|>=1.6,<2.0|>=2.0.0|>=2.2.0|>=2.2.0,<3|>=2.3.0,<2.4|>=2.4.0,<2.5|>=2.4.0,<=2.5']
python-language-server

In [84]:
conda install -c conda-forge geopandas


Note: you may need to restart the kernel to use updated packages.Collecting package metadata (current_repodata.json): ...working... done



Examining qtpy:   2%|2         | 7/310 [02:12<2:00:55, 23.95s/it]      


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed



Examining @/win-64::__archspec==1=x86_64:  32%|###1      | 99/310 [27:32<45:35, 12.97s/it]  


Examining html5lib:  52%|#####1    | 161/310 [57:19<20:19,  8.19s/it]  


...

conda-verify -> pyyaml -> cython
bokeh -> pyyaml[version='>=3.10'] -> cython
dask-core -> pyyaml -> cython
anaconda-client -> pyyaml[version='>=3.12'] -> cython

Package send2trash conflicts for:
anaconda==2020.11 -> send2trash==1.5.0[build='py38_0|py37_0']
widgetsnbextension -> notebook[version='>=4.4.1'] -> send2trash[version='>=1.5.0']
jupyter -> notebook -> send2trash[version='>=1.5.0']
jupyterlab_server -> jupyter_server[version='>=1.1.0,<2.0.0'] -> send2trash[version='>=1.5.0']
notebook -> send2trash[version='>=1.5.0']
ipywidgets -> notebook -> send2trash[version='>=1.5.0']
anaconda==2020.11 -> notebook==6.1.4=py37_0 -> send2trash
jupyterlab -> jupyter_server[version='>=1.4,<2'] -> send2trash[version='>=1.5.0']

Package sphinxcontrib-devhelp conflicts for:
anaconda==2020.11 -> sphinxcontrib-devhelp==1.0.2=py_0
spyder -> sphinx[version='>=0.6.6'] -> sphinxcontrib-devhelp
anaconda==2020.11 -> sphinx==3.2.1=py_0 -> sphinxcontrib-devhelp
sphinx -> sphinxcontrib-devhelp
numpydoc -> sp

jupyterlab_server -> jsonschema[version='>=3.0.1'] -> functools32
geopandas -> matplotlib-base -> functools32
jsonschema -> functools32

Package libssh2 conflicts for:
libcurl -> libssh2[version='>=1.8.0,<2.0.0a0|>=1.9.0,<2.0a0|>=1.8.2,<2.0a0|>=1.8.0,<2.0a0']
hdf5 -> libcurl[version='>=7.71.1,<8.0a0'] -> libssh2[version='>=1.9.0,<2.0a0']
anaconda==2020.11 -> curl==7.71.1=h2a8f88b_1 -> libssh2[version='>=1.9.0,<2.0a0']
pycurl -> libcurl[version='>=7.71.1,<8.0a0'] -> libssh2[version='1.*|>=1.8.0,<2.0.0a0|>=1.9.0,<2.0a0|>=1.8.2,<2.0a0|>=1.8.0,<2.0a0']
anaconda==2020.11 -> libssh2==1.9.0=h7a1dbc1_1
curl -> libssh2[version='1.*|>=1.8.0,<2.0.0a0|>=1.9.0,<2.0a0|>=1.8.2,<2.0a0|>=1.8.0,<2.0a0']

Package pywin32-ctypes conflicts for:
keyring -> pywin32-ctypes[version='!=0.1.0,!=0.1.1']
anaconda==2020.11 -> pywin32-ctypes==0.2.0[build='py38_1000|py37_1001']
anaconda==2020.11 -> keyring==21.4.0=py37_1 -> pywin32-ctypes
spyder -> keyring[version='>=17.0.0'] -> pywin32-ctypes[version='!=0.1.0,!=0.1.

Package tk conflicts for:
krb5 -> tk[version='8.6.*|>=8.6.10,<8.7.0a0|>=8.6.9,<8.7.0a0|>=8.6.8,<8.7.0a0|>=8.6.7,<8.7.0a0']
anaconda-navigator -> pillow -> tk[version='8.6.*|>=8.6.10,<8.7.0a0|>=8.6.9,<8.7.0a0|>=8.6.8,<8.7.0a0|>=8.6.7,<8.7.0a0']
bokeh -> pillow[version='>=7.1.0'] -> tk[version='8.6.*|>=8.6.10,<8.7.0a0|>=8.6.8,<8.7.0a0|>=8.6.9,<8.7.0a0|>=8.6.7,<8.7.0a0']
imageio -> pillow -> tk[version='8.6.*|>=8.6.10,<8.7.0a0|>=8.6.9,<8.7.0a0|>=8.6.8,<8.7.0a0|>=8.6.7,<8.7.0a0']
pillow -> tk[version='8.6.*|>=8.6.10,<8.7.0a0|>=8.6.9,<8.7.0a0|>=8.6.8,<8.7.0a0|>=8.6.7,<8.7.0a0']
scikit-image -> pillow[version='>=4.3.0'] -> tk[version='8.6.*|>=8.6.10,<8.7.0a0|>=8.6.9,<8.7.0a0|>=8.6.8,<8.7.0a0|>=8.6.7,<8.7.0a0']
matplotlib-base -> pillow[version='>=6.2.0'] -> tk[version='>=8.6.10,<8.7.0a0|>=8.6.9,<8.7.0a0|>=8.6.8,<8.7.0a0']
curl -> krb5[version='>=1.17.1,<1.18.0a0'] -> tk[version='8.6.*|>=8.6.10,<8.7.0a0|>=8.6.8,<8.7.0a0|>=8.6.9,<8.7.0a0|>=8.6.7,<8.7.0a0']
anaconda==2020.11 -> tk==8.6.10=he774

anaconda==2020.11 -> pyqt==5.9.2=py37h6538335_2 -> sip[version='>=4.19.13,<=4.19.14|>=4.19.4,<=4.19.8']

Package joblib conflicts for:
nltk -> scikit-learn -> joblib[version='>=0.11']
nltk -> joblib
scikit-learn -> joblib[version='>=0.11']
anaconda==2020.11 -> nltk==3.5=py_0 -> joblib[version='>=0.11']
anaconda==2020.11 -> joblib==0.17.0=py_0

Package testpath conflicts for:
spyder -> nbconvert[version='>=4.0'] -> testpath
notebook -> nbconvert -> testpath
nbconvert -> testpath
anaconda==2020.11 -> nbconvert==6.0.7=py37_0 -> testpath
jupyter -> nbconvert -> testpath
anaconda==2020.11 -> testpath==0.4.4=py_0

Package brotlipy conflicts for:
requests -> urllib3[version='>=1.21.1,<1.27'] -> brotlipy[version='>=0.6.0']
anaconda==2020.11 -> brotlipy==0.7.0[build='py37he774522_1000|py38he774522_1000']
anaconda==2020.11 -> urllib3==1.25.11=py_0 -> brotlipy[version='>=0.6.0']
urllib3 -> brotlipy[version='>=0.6.0']

Package pympler conflicts for:
pytest -> attrs[version='>=17.2.0'] -> pympler
s


Package lcms2 conflicts for:
scikit-image -> pillow[version='>=4.3.0'] -> lcms2[version='>=2.12,<3.0a0']
imageio -> pillow -> lcms2[version='>=2.12,<3.0a0']
matplotlib-base -> pillow[version='>=6.2.0'] -> lcms2[version='>=2.12,<3.0a0']
bokeh -> pillow[version='>=7.1.0'] -> lcms2[version='>=2.12,<3.0a0']
pillow -> lcms2[version='>=2.12,<3.0a0']
tifffile -> imagecodecs[version='>=2021.1.11'] -> lcms2[version='>=2.11,<3.0a0|>=2.12,<3.0a0|>=2.9,<3.0a0']
anaconda-navigator -> pillow -> lcms2[version='>=2.12,<3.0a0']

Package hdf5 conflicts for:
anaconda==2020.11 -> hdf5==1.10.4=h7ebc959_0
pytables -> hdf5[version='1.10.1.*|1.10.1|>=1.10.3,<1.10.4.0a0|>=1.10.4,<1.10.5.0a0|>=1.10.5,<1.10.6.0a0|>=1.10.6,<1.10.7.0a0|>=1.10.2,<1.10.3.0a0|1.8.18|1.8.18.*|1.8.17|1.8.17.*|>=1.8.18,<1.8.19.0a0|>=1.8.18,<1.9.0a0|>=1.10.1,<1.10.2.0a0']
anaconda==2020.11 -> h5py==2.10.0=py37h5e291fa_0 -> hdf5[version='>=1.10.4,<1.10.5.0a0']
h5py -> hdf5[version='1.10.1|1.10.1.*|>=1.10.2,<1.10.3.0a0|>=1.10.3,<1.10.4.0a

In [87]:
pip install gdal

  Using cached GDAL-3.2.1.tar.gz (604 kB)
  Running setup.py clean for gdal
Failed to build gdal
    Running setup.py install for gdal: started
    Running setup.py install for gdal: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\mikha\Anaconda\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\mikha\\AppData\\Local\\Temp\\pip-install-9zuc7h0j\\gdal\\setup.py'"'"'; __file__='"'"'C:\\Users\\mikha\\AppData\\Local\\Temp\\pip-install-9zuc7h0j\\gdal\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\mikha\AppData\Local\Temp\pip-wheel-1srfqk5n'
       cwd: C:\Users\mikha\AppData\Local\Temp\pip-install-9zuc7h0j\gdal\
  Complete output (54 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\osgeo
  copying osgeo\gdal.py -> build\lib.win-amd64-3.8\osgeo
  copying osgeo\gdalconst.py -> build\lib.win-amd64-3.8\osgeo
  copying osgeo\gdalnumeric.py -> build\lib.win-amd64-3.8\os

In [86]:
pip install geopandas

  Using cached geopandas-0.8.2-py2.py3-none-any.whl (962 kB)
  Using cached Fiona-1.8.18.tar.gz (1.3 MB)
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\mikha\Anaconda\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\mikha\\AppData\\Local\\Temp\\pip-install-iepe1w6o\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\mikha\\AppData\\Local\\Temp\\pip-install-iepe1w6o\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\mikha\AppData\Local\Temp\pip-pip-egg-info-qwaypcxs'
         cwd: C:\Users\mikha\AppData\Local\Temp\pip-install-iepe1w6o\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command outp

## US County Shapefiles

# Gameplan

* Get county shapefiles
* Get congressional district shapefiles
* map counties (and county-data I have) to their districts in a dataframe